In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os

file_path = "../../datasets/original_treated.csv"

try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Erro: Arquivo '{file_path}' não encontrado.")
    print(f"Verifique se o arquivo está no local correto. Diretorio atual: {os.getcwd()}")

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

# target
target = 'is_approved'

# features
features = [
    'g1',
    'g2',
    'final_grade',
    'sex',
    'week_day',
    'period',
    'class_skips'
]

missing_features = [col for col in features if col not in df.columns]
if missing_features:
    print(f"Erro: As seguintes colunas de features não foram encontradas no CSV: {missing_features}")
    print("\nColunas disponíveis no CSV:")
    print(list(df.columns))
    exit()

print(f"Usando '{target}' como alvo e {len(features)} features.")

X = df[features]
y = df[target]

X = X.fillna(0)

# divisão treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,    # 30% para teste
    random_state=42,  # Para reprodutibilidade
    stratify=y        # Mantém a proporção de 0s e 1s
)

print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

# criação do modelo RandomForest
rf_model = RandomForestClassifier(
    n_estimators=500, 
    random_state=42, 
    n_jobs=-1
)

print("Iniciando treinamento do RandomForest...")
rf_model.fit(X_train, y_train)
print("Treinamento concluído.")

# previsão e avaliação
y_pred = rf_model.predict(X_test)

# acurácia
accuracy = accuracy_score(y_test, y_pred)
print("-" * 30)
print(f"Acurácia do Modelo: {accuracy * 100:.2f}%")
print("-" * 30)

# matriz de confusão
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\n(Linhas = Real, Colunas = Previsto)")
print("-" * 30)

# relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Não Aprovado (0)", "Aprovado (1)"]))
print("-" * 30)

# importância das features
feature_importances = pd.Series(rf_model.feature_importances_, index=features)
feature_importances = feature_importances.sort_values(ascending=False)

print("Importância das Features:")
print(feature_importances)

Dataset carregado com 35656 linhas e 17 colunas.
Usando 'is_approved' como alvo e 7 features.
Dados divididos: 24959 para treino, 10697 para teste.
Iniciando treinamento do RandomForest...
Treinamento concluído.
------------------------------
Acurácia do Modelo: 99.88%
------------------------------
Matriz de Confusão:
[[2542   12]
 [   1 8142]]

(Linhas = Real, Colunas = Previsto)
------------------------------
Relatório de Classificação:
                  precision    recall  f1-score   support

Não Aprovado (0)       1.00      1.00      1.00      2554
    Aprovado (1)       1.00      1.00      1.00      8143

        accuracy                           1.00     10697
       macro avg       1.00      1.00      1.00     10697
    weighted avg       1.00      1.00      1.00     10697

------------------------------
Importância das Features (Quais colunas mais influenciam):
final_grade    0.432382
g2             0.297406
g1             0.230856
class_skips    0.025380
period         0.01